# Data Preparation and Cleaning

The data used in this report comes from two sources: 
- Pollen data comes from The <a href="https://www.houstontx.gov/health/Pollen-Mold/pollen-archives.html">Houston Health Department website</a>
- Climate data comes from the the <a href="ncei.noaa.gov"> National Oceanic and Atmospheric Administration's (NOAA) National Center for Environmental Information (NCEI) </a>

Pollen Data was compiled from individual monthly spreadsheets from 2013 to 2022 consisting of pollen counts 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

Having combined the multiple spreadsheets in Excel, the resulting spreadsheet is loaded below:

In [5]:
pollen_df = pd.read_csv('C://Users/Nick/Documents/Flatiron/capstone/data/all_pollen.csv')

In [6]:
pollen_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3377 entries, 0 to 3376
Data columns (total 70 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   DATE                         3377 non-null   object 
 1   Alder                        1068 non-null   float64
 2   Ash                          2358 non-null   float64
 3   Ashe Juniper / Bald Cypress  2358 non-null   float64
 4   Black Gum                    2358 non-null   float64
 5   Black Walnut                 2342 non-null   float64
 6   Bushes                       2358 non-null   float64
 7   Birch                        2358 non-null   float64
 8   Cedar                        832 non-null    float64
 9   Cotton Wood                  2358 non-null   float64
 10  Dogwood                      2358 non-null   float64
 11  Elm                          2358 non-null   float64
 12  Glandular Mesquite           2358 non-null   float64
 13   Hackberry        

In [7]:

pollen_df['DATE'] = pd.to_datetime(pollen_df['DATE'])

In [8]:
#some of the text data
pollen_df['DATE'] = pd.to_numeric(pollen_df['DATE'], errors = 'coerce')

In [9]:
pollen_df['DATE'] = pd.to_datetime(pollen_df['DATE'])

In [10]:
#setting the index as date time
pollen_df.set_index('DATE', drop=True, inplace=True)

In [11]:
#sorting the date index so it works well in our further manipulations
pollen_df = pollen_df.sort_index()

In [12]:
pollen_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3377 entries, 2013-01-01 to 2022-03-31
Data columns (total 69 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Alder                        1068 non-null   float64
 1   Ash                          2358 non-null   float64
 2   Ashe Juniper / Bald Cypress  2358 non-null   float64
 3   Black Gum                    2358 non-null   float64
 4   Black Walnut                 2342 non-null   float64
 5   Bushes                       2358 non-null   float64
 6   Birch                        2358 non-null   float64
 7   Cedar                        832 non-null    float64
 8   Cotton Wood                  2358 non-null   float64
 9   Dogwood                      2358 non-null   float64
 10  Elm                          2358 non-null   float64
 11  Glandular Mesquite           2358 non-null   float64
 12   Hackberry                   2358 non-null   float64
 13  

As evindenced above, there are many missing values. The Technicians who measured pollen counts do not work on weekends or holidays. This is address further down in the cleaning. 

In [13]:
#all columns should be  numeric exepct tech
pollen_df['Willow'] = pd.to_numeric(pollen_df['Willow'], errors='coerce') 

pollen_df['Gingko Biloba'] = pd.to_numeric(pollen_df['Gingko Biloba'], errors='coerce') 

pollen_df['Privet'] = pd.to_numeric(pollen_df['Privet'], errors='coerce') 

In [14]:
! mkdir data

A subdirectory or file data already exists.


In [15]:
#
pollen_df.to_csv('data/pollen_df.csv')

### preliminary exploration 
taking a look at initial correlations to see the landscape of the data:

### Pulling in Data from Shreveport
Shreveport, Lousiana is located just over 200 miles fro Houston beyond a lusher forested area. We would like to see if weather indicators there have any greater or lesser impact on pollen counts than that of Houston proper.

In [16]:
shreveport_df = pd.read_csv('data/shreveport.csv')
shreveport_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3372 entries, 0 to 3371
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DATE    3372 non-null   object 
 1   AWND    3350 non-null   float64
 2   PRCP    3372 non-null   float64
 3   TAVG    3372 non-null   float64
 4   WDF2    3372 non-null   int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 131.8+ KB


In [17]:
#converting the 'DATE' column to the correct format
shreveport_df['DATE'] = pd.to_datetime(shreveport_df['DATE'])

In [18]:
combined_df = pollen_df.merge(shreveport_df, on='DATE')

In [19]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3372 entries, 0 to 3371
Data columns (total 74 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   DATE                         3372 non-null   datetime64[ns]
 1   Alder                        1064 non-null   float64       
 2   Ash                          2354 non-null   float64       
 3   Ashe Juniper / Bald Cypress  2354 non-null   float64       
 4   Black Gum                    2354 non-null   float64       
 5   Black Walnut                 2338 non-null   float64       
 6   Bushes                       2354 non-null   float64       
 7   Birch                        2354 non-null   float64       
 8   Cedar                        828 non-null    float64       
 9   Cotton Wood                  2354 non-null   float64       
 10  Dogwood                      2354 non-null   float64       
 11  Elm                          2354 non-null 

The number of columns is excessive so we remove snow and just keep averages of wind speed and temperature for which there are stronger correlations to pollen counts anyway as shown below:

In [20]:
combined_df = combined_df.drop(['SNOW_60', 'SNWD_60', 'TMAX_60', 'TMIN_60', 'WDF2_60', 'WDF5_60', 'WSF2_60', 'WSF5_60',
                                'SNOW_18', 'SNWD_18', 'TMAX_18','TMIN_18', 'WDF2_18', 'WDF5_18', 'WSF2_18', 'WSF5_18'], axis=1)

In [21]:
combined_df.columns

Index(['DATE', 'Alder', 'Ash', 'Ashe Juniper / Bald Cypress', 'Black Gum ',
       'Black Walnut', 'Bushes', 'Birch', 'Cedar', 'Cotton Wood', 'Dogwood',
       'Elm', 'Glandular Mesquite', ' Hackberry', 'Hickory', 'Mulberry',
       'Maple', 'Osage Orange', 'Oak', 'Sycamore', 'Pine', 'Privet',
       'Sweet Gum', 'Gingko Biloba', 'Magnolia', 'Willow', 'Tree Total',
       'Grass Total', 'Tree & Grass Total', 'Amaranth', 'Burweed / Marshelder',
       'Cattail', 'Dog Fennel', 'Lamb's Quarters', 'Nettle', 'Partridge Pea',
       'Pigweed', 'Plum Grannet', 'Ragweed', 'Rumex', 'Sagebrush', 'Saltbrush',
       'Sedge', 'Sneezeweed', 'Other Weed', 'Weed Total', 'Pollen Total',
       'Tech.', 'AWND_60', 'PRCP_60', 'TAVG_60', 'AWND_18', 'PRCP_18',
       'TAVG_18', 'AWND', 'PRCP', 'TAVG', 'WDF2'],
      dtype='object')

In [22]:
# the merge changed our index so we reset to datetime
combined_df.set_index('DATE', inplace=True, drop=True)

In [23]:
combined_df.sort_index(inplace=True)

In [24]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3372 entries, 2013-01-01 to 2022-03-31
Data columns (total 57 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Alder                        1064 non-null   float64
 1   Ash                          2354 non-null   float64
 2   Ashe Juniper / Bald Cypress  2354 non-null   float64
 3   Black Gum                    2354 non-null   float64
 4   Black Walnut                 2338 non-null   float64
 5   Bushes                       2354 non-null   float64
 6   Birch                        2354 non-null   float64
 7   Cedar                        828 non-null    float64
 8   Cotton Wood                  2354 non-null   float64
 9   Dogwood                      2354 non-null   float64
 10  Elm                          2354 non-null   float64
 11  Glandular Mesquite           2354 non-null   float64
 12   Hackberry                   2354 non-null   float64
 13  

In [25]:
# for clarity's sake change the names of the columns with information from shreveport by adding the suffix _S
combined_df[['AWND_S', 'PRCP_S', 'TAVG_S', 'WDF2_S']] = combined_df[['AWND', 'PRCP', 'TAVG', 'WDF2']]

In [26]:
combined_df = combined_df.drop(columns=['AWND', 'PRCP', 'TAVG', 'WDF2'])

It was decided to preserve a workable record of the data before imputing missing values to compare to the imputed data. This was done by upsampling the time series to weekly.

In [27]:
weekly_raw = combined_df.resample('W').mean()

# Feature Engineering and Imputing missing values
Weed, grass and tree pollens are interpreted differently in terms of severity. For instance, a 200 is a high count for grass pollen, and 1000 is a high count for tree pollen. Given that this is a classificcation project, binary columns for high and very high pollen counts are created. More information is given at the <a href= 'https://www.houstontx.gov/health/Pollen-Mold/numbers.html'>Houston Health Dept. website.</a>

In [28]:
#first missing values are interpolated linearly. Pollen Total in included to compare to the weighted pollen count
for total in ['Tree Total', 'Grass Total', 'Weed Total', 'Pollen Total']:
    combined_df[total] = combined_df[total].interpolate()

In [29]:
#feature engineering a weighted pollen column that accounts for severity of different
combined_df['pollen_weighted'] = combined_df['Tree Total'] + combined_df['Grass Total']*7.5 +combined_df['Weed Total']*3
weekly_raw['pollen_weighted'] = weekly_raw['Tree Total'] + weekly_raw['Grass Total']*7.5 + weekly_raw['Weed Total']*3

pollen counts above have been weighted based on "extremely heavy" rating for trees of 1500+ so this is chosen as the threshold for `super_high` pollen count in the classification columns. We assign ratings of very_high as somewhere in the middle, and high as the lower threshold for "heavy" pollen counts as follows

In [30]:
combined_df['super_high'] = (combined_df['pollen_weighted'] > 1500).astype(int)
combined_df['very_high'] = (combined_df['pollen_weighted'] > 500).astype(int)
combined_df['high'] = (combined_df['pollen_weighted']>100).astype(int)

weekly_raw['super_high'] = (weekly_raw['pollen_weighted'] > 1500).astype(int)
weekly_raw['very_high'] = (weekly_raw['pollen_weighted'] > 500).astype(int)
weekly_raw['high'] = (weekly_raw['pollen_weighted']>100).astype(int)

In [31]:
#checking class balances for each category for interpolated data
for lvl in ['high', 'very_high', 'super_high']:
    print(combined_df[lvl].value_counts()) 
    print(combined_df[lvl].value_counts(normalize=True))

1    1771
0    1601
Name: high, dtype: int64
1    0.525208
0    0.474792
Name: high, dtype: float64
0    2697
1     675
Name: very_high, dtype: int64
0    0.799822
1    0.200178
Name: very_high, dtype: float64
0    3151
1     221
Name: super_high, dtype: int64
0    0.93446
1    0.06554
Name: super_high, dtype: float64


In [32]:
#checking class balances for each category for non-imputed data
for lvl in ['high', 'very_high', 'super_high']:
    print(weekly_raw[lvl].value_counts()) 
    print(weekly_raw[lvl].value_counts(normalize=True))

1    280
0    203
Name: high, dtype: int64
1    0.57971
0    0.42029
Name: high, dtype: float64
0    379
1    104
Name: very_high, dtype: int64
0    0.784679
1    0.215321
Name: very_high, dtype: float64
0    455
1     28
Name: super_high, dtype: int64
0    0.942029
1    0.057971
Name: super_high, dtype: float64


In comparing the breakdowns of our imputed and unimputed data, the class breakdown for each level of severity is roughly the same. This provides enough confidence that the imputed data can be used moving forward. We name it and save it as a new master dataset below

In [33]:
master_df = combined_df.copy()

In [34]:
master_df = master_df.asfreq('D')

In [35]:
master_df.to_csv('data/master_df.csv')